In [19]:
import torch
import yfinance as yf
import numpy as np
import paddle
import numpy as np
np.object = object
from paddle_quantum.ansatz import Circuit
from paddle_quantum.gate import BasisEncoding, AmplitudeEncoding, AngleEncoding, IQPEncoding
import paddle_quantum as pq
import matplotlib.pyplot as plt

In [ ]:
sp500 = yf.download('^GSPC', start='2000-01-01', end='2023-12-31')
prices = sp500['Adj Close']

# Preprocess the data: calculate returns or normalize directly
log_returns = np.log(prices / prices.shift(1)).dropna()
normalized_data = (log_returns - log_returns.mean()) / log_returns.std()

# Convert to a numpy array for training
training_data = normalized_data.values

qubits = 13

[*********************100%***********************]  1 of 1 completed


In [29]:
amplitude_enc = AmplitudeEncoding(num_qubits=qubits)
x = paddle.to_tensor(training_data)
state = amplitude_enc(x)
print(state)


[-0.04086019+0.j  0.00179228+0.j  0.00078874+0.j ...  0.        +0.j
  0.        +0.j  0.        +0.j]
